In [ ]:
# ============================================================================
# FX Skew Divergence Signal — Per-Pair Tuned Evaluation
# ============================================================================
# Signal: Short-dated rho moved while longer-dated rho is quiet.
# Key insight: optimal tenor pair differs by pair liquidity:
#   EUR (most liquid) → 1W vs 1M    GBP (liquid) → 1M vs 3M
#   JPY (USD/xxx)     → 1M vs 3M (7-day lookback, bear direction)
# ============================================================================

import sys, warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
sys.path.insert(0, str(Path('.').resolve()))

from scripts.fx_skew_divergence import (
    CURRENCIES, PAIR_MAP, PAIR_CONFIGS, SPOT_SIGN, FORWARD_HORIZONS,
    load_pair_data, evaluate_pair, run_backtest
)

# --- Load all pairs with their tuned tenor configs ---
data = {}
for ccy in CURRENCIES:
    cfg = PAIR_CONFIGS[ccy]
    df, mask, info = load_pair_data(ccy)
    data[ccy] = {'df': df, 'mask': mask, 'info': info}
    print(f"{ccy} ({info['pair']}): {cfg['fast_tenor']}→{cfg['slow_tenor']} | "
          f"{info['n_overlap']:>3d} overlap dates | "
          f"{info['date_min'].date()} to {info['date_max'].date()} | "
          f"boundary: {info['n_boundary']} ({info['boundary_pct']:.1f}%)")

print('\nPer-pair configs:')
for ccy in CURRENCIES:
    cfg = PAIR_CONFIGS[ccy]
    print(f"  {PAIR_MAP[ccy]}: {cfg['fast_tenor']}→{cfg['slow_tenor']}, "
          f"fast={cfg['fast_method']}{cfg['fast_window']}d q{cfg['fast_threshold_q']}, "
          f"slow={cfg['slow_method']}{cfg['slow_window']}d quiet@{cfg['quiet_q']}, "
          f"hold={cfg['hold_days']}d, spot_sign={SPOT_SIGN[ccy]:+d}")

In [ ]:
# ============================================================================
# Cell 2: Signal Evaluation — Per Pair (tuned configs)
# ============================================================================

results = {}
for ccy in CURRENCIES:
    results[ccy] = evaluate_pair(data[ccy]['df'], data[ccy]['mask'], ccy)

# --- Summary table ---
rows = []
for ccy in CURRENCIES:
    r = results[ccy]
    cfg = PAIR_CONFIGS[ccy]
    si = r['signal_info']
    for direction, label in [('bull_div', 'Bull'), ('bear_div', 'Bear')]:
        for h in ['5d', '10d']:
            ev = r[direction].get(h, {})
            rows.append({
                'Pair': r['pair'],
                'Tenors': f"{cfg['fast_tenor']}→{cfg['slow_tenor']}",
                'Dir': label,
                'Horizon': h,
                'n': ev.get('n', 0),
                'Mean (bps)': ev.get('mean_bps', np.nan),
                'Hit %': ev.get('hit_rate', np.nan) * 100 if ev.get('hit_rate') else np.nan,
                't-stat': ev.get('t_stat', np.nan),
                'p-value': ev.get('p_value', np.nan),
            })

summary = pd.DataFrame(rows)
print('DIVERGENCE SIGNAL — PER-PAIR TUNED')
print('=' * 90)
print(summary.to_string(index=False, float_format=lambda x: f'{x:.2f}'))

# --- Ablation: fast-only (no quiet filter) ---
print('\n\nABLATION: FAST-ONLY (no quiet filter)')
print('=' * 90)
rows_abl = []
for ccy in CURRENCIES:
    r = results[ccy]
    cfg = PAIR_CONFIGS[ccy]
    for direction, label in [('bull_fast_only', 'Bull'), ('bear_fast_only', 'Bear')]:
        for h in ['5d', '10d']:
            ev = r[direction].get(h, {})
            rows_abl.append({
                'Pair': r['pair'],
                'Tenors': f"{cfg['fast_tenor']}→{cfg['slow_tenor']}",
                'Dir': label, 'Horizon': h,
                'n': ev.get('n', 0),
                'Mean (bps)': ev.get('mean_bps', np.nan),
                'Hit %': ev.get('hit_rate', np.nan) * 100 if ev.get('hit_rate') else np.nan,
                't-stat': ev.get('t_stat', np.nan),
                'p-value': ev.get('p_value', np.nan),
            })
print(pd.DataFrame(rows_abl).to_string(index=False, float_format=lambda x: f'{x:.2f}'))

# --- Quiet filter value-add ---
print('\n\nQUIET FILTER VALUE-ADD (divergence t minus fast-only t, 5d + 10d)')
print('-' * 70)
for ccy in CURRENCIES:
    r = results[ccy]
    for d_div, d_fo, label in [('bull_div', 'bull_fast_only', 'Bull'), ('bear_div', 'bear_fast_only', 'Bear')]:
        for h in ['5d', '10d']:
            t_div = r[d_div].get(h, {}).get('t_stat', np.nan)
            t_fo = r[d_fo].get(h, {}).get('t_stat', np.nan)
            delta = t_div - t_fo if not (np.isnan(t_div) or np.isnan(t_fo)) else np.nan
            if not np.isnan(delta):
                print(f"  {r['pair']} {label:4s} {h}: div t={t_div:+.2f}, fast-only t={t_fo:+.2f}, delta={delta:+.2f}")

In [ ]:
# ============================================================================
# Cell 3: Cross-Pair Comparison + Statistical Rigour
# ============================================================================

from scipy import stats as sp_stats

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
pairs = [PAIR_MAP[c] for c in CURRENCIES]

for ax_idx, (h, ax) in enumerate(zip(['5d', '10d'], axes)):
    bull_t = [results[c]['bull_div'].get(h, {}).get('t_stat', 0) for c in CURRENCIES]
    bear_t = [results[c]['bear_div'].get(h, {}).get('t_stat', 0) for c in CURRENCIES]
    x = np.arange(len(pairs))
    w = 0.35
    ax.bar(x - w/2, bull_t, w, label='Bull', color='steelblue', alpha=0.8)
    ax.bar(x + w/2, bear_t, w, label='Bear', color='salmon', alpha=0.8)
    ax.axhline(1.96, color='red', ls='--', lw=1, label='p<0.05')
    ax.axhline(-1.96, color='red', ls='--', lw=1)
    ax.axhline(0, color='grey', lw=0.5)
    ax.set_xticks(x)
    ax.set_xticklabels([f"{p}\n({PAIR_CONFIGS[c]['fast_tenor']}→{PAIR_CONFIGS[c]['slow_tenor']})" for c, p in zip(CURRENCIES, pairs)], fontsize=9)
    ax.set_ylabel('t-statistic')
    ax.set_title(f'{h} Forward Returns')
    ax.legend(fontsize=8)

plt.suptitle('Divergence Signal: t-statistics by Pair (tuned tenor pairs)', fontweight='bold')
plt.tight_layout()
plt.show()

# --- Sub-period consistency ---
print('\nSUB-PERIOD CONSISTENCY (bps)')
print('=' * 80)
print(f'{"Pair":<10} {"Dir":<6} {"Hz":<4} {"H1 (bps)":>10} {"H2 (bps)":>10} {"Consistent?":>12}')
print('-' * 80)
for ccy in CURRENCIES:
    r = results[ccy]
    for label in ['bull', 'bear']:
        for h in ['5d', '10d']:
            h1 = r.get(f'{label}_H1_{h}_bps', np.nan)
            h2 = r.get(f'{label}_H2_{h}_bps', np.nan)
            con = r.get(f'{label}_{h}_consistent', False)
            h1_s = f'{h1:>+10.1f}' if not np.isnan(h1) else '       N/A'
            h2_s = f'{h2:>+10.1f}' if not np.isnan(h2) else '       N/A'
            print(f'{r["pair"]:<10} {label.title():<6} {h:<4} {h1_s} {h2_s} {"YES" if con else "NO":>12}')

# --- Bonferroni correction ---
n_tests = len(CURRENCIES) * 2 * 2  # pairs * directions * horizons
bonferroni_threshold = 0.05 / n_tests

print(f'\n\nMULTIPLE TESTING CORRECTION')
print(f'Total tests: {n_tests} | Bonferroni threshold: p < {bonferroni_threshold:.4f}')
print('-' * 80)
any_survives = False
for ccy in CURRENCIES:
    r = results[ccy]
    for direction, label in [('bull_div', 'Bull'), ('bear_div', 'Bear')]:
        for h in ['5d', '10d']:
            p = r[direction].get(h, {}).get('p_value', 1.0)
            t = r[direction].get(h, {}).get('t_stat', 0)
            n = r[direction].get(h, {}).get('n', 0)
            survives = isinstance(p, float) and p < bonferroni_threshold
            if survives:
                any_survives = True
            marker = ' ** SURVIVES **' if survives else ''
            print(f'  {r["pair"]} {label:4s} {h}: t={t:+.2f}, p={p:.4f}, n={n}{marker}')

if not any_survives:
    print('\n  >> No individual test survives Bonferroni correction.')

# --- Forward horizon profiles for the best direction per pair ---
print('\n\nFORWARD RETURN PROFILES (best direction per pair)')
print('=' * 80)
# Identify best direction per pair
for ccy in CURRENCIES:
    r = results[ccy]
    cfg = PAIR_CONFIGS[ccy]
    ss = SPOT_SIGN[ccy]
    
    # Find best direction/horizon combo
    best_t, best_dir, best_mask, best_sign = 0, None, None, None
    for d, m_key, sign in [('bull_div', 'bull_mask', +ss), ('bear_div', 'bear_mask', -ss)]:
        for h in ['5d', '10d']:
            t_val = abs(r[d].get(h, {}).get('t_stat', 0))
            if t_val > best_t:
                best_t = t_val
                best_dir = d.split('_')[0]
                best_mask = r[m_key]
                best_sign = sign
    
    print(f'\n  {r["pair"]} ({cfg["fast_tenor"]}→{cfg["slow_tenor"]}) — {best_dir} direction:')
    print(f'  {"Hz":>5} {"n":>4} {"bps":>8} {"hit%":>6} {"t":>7} {"p":>8}')
    for hz in FORWARD_HORIZONS:
        rets = data[ccy]['df'].loc[best_mask, f'fwd_ret_{hz}d'].dropna() * best_sign
        n = len(rets)
        if n >= 3:
            t, p = sp_stats.ttest_1samp(rets, 0)
            print(f'  {hz:>4}d {n:>4} {rets.mean()*10000:>+8.1f} {(rets>0).mean()*100:>5.0f}% {t:>+7.2f} {p:>8.4f}')

In [ ]:
# ============================================================================
# Cell 4: Backtest Per Pair
# ============================================================================

bt_results = {}
for ccy in CURRENCIES:
    r = results[ccy]
    cfg = PAIR_CONFIGS[ccy]
    bt_results[ccy] = run_backtest(
        data[ccy]['df'], r['bull_mask'], r['bear_mask'],
        hold_days=cfg['hold_days']
    )

# --- Backtest summary table ---
print('BACKTEST SUMMARY (per-pair tuned, 0.5bp cost per trade)')
print('=' * 90)
print(f'{"Pair":<10} {"Tenors":<8} {"Trades":>7} {"Active%":>8} {"SR Gross":>9} {"SR Net":>9} '
      f'{"Ret Gross%":>11} {"Ret Net%":>9} {"Max DD%":>8}')
print('-' * 90)
for ccy in CURRENCIES:
    bt = bt_results[ccy]
    cfg = PAIR_CONFIGS[ccy]
    pct_active = bt['active_days'] / bt['total_days'] * 100
    tenors = f"{cfg['fast_tenor']}→{cfg['slow_tenor']}"
    print(f"{PAIR_MAP[ccy]:<10} {tenors:<8} {bt['n_trades']:>7} {pct_active:>7.1f}% {bt['sharpe_gross']:>9.2f} "
          f"{bt['sharpe_net']:>9.2f} {bt['total_return_gross']:>10.1f}% {bt['total_return_net']:>8.1f}% "
          f"{bt['max_drawdown']:>7.1f}%")

# --- Equity curves ---
fig, axes = plt.subplots(1, len(CURRENCIES), figsize=(5*len(CURRENCIES), 4), sharey=False)
for ax, ccy in zip(axes, CURRENCIES):
    bt = bt_results[ccy]['bt']
    cfg = PAIR_CONFIGS[ccy]
    ax.plot(bt['date'], bt['cum_gross'], label='Gross', color='steelblue', lw=1.2)
    ax.plot(bt['date'], bt['cum_net'], label='Net', color='darkblue', lw=1.2)
    ax.axhline(1, color='grey', lw=0.5)
    tenors = f"{cfg['fast_tenor']}→{cfg['slow_tenor']}"
    ax.set_title(f"{PAIR_MAP[ccy]} {tenors} (SR={bt_results[ccy]['sharpe_net']:.2f})")
    ax.legend(fontsize=8)
    ax.tick_params(axis='x', rotation=45)

plt.suptitle('Equity Curves — Per-Pair Tuned Divergence Signal', fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================================
# Cell 5: Honest Assessment
# ============================================================================

from scipy import stats as sp_stats

print('='*80)
print('HONEST ASSESSMENT — FX SKEW DIVERGENCE SIGNAL (Per-Pair Tuned)')
print('='*80)

# 1. Per-pair verdict
print('\n1. PER-PAIR VERDICT')
print('-'*70)

for ccy in CURRENCIES:
    r = results[ccy]
    cfg = PAIR_CONFIGS[ccy]
    si = r['signal_info']
    bt = bt_results[ccy]
    ss = SPOT_SIGN[ccy]
    
    combos = []
    for d, label in [('bull_div', 'Bull'), ('bear_div', 'Bear')]:
        for h in ['5d', '10d']:
            ev = r[d].get(h, {})
            combos.append({
                'dir': label, 'h': h,
                't': ev.get('t_stat', 0),
                'hit': ev.get('hit_rate', 0) or 0,
                'n': ev.get('n', 0),
                'p': ev.get('p_value', 1.0),
                'bps': ev.get('mean_bps', 0),
            })
    
    best = max(combos, key=lambda x: abs(x['t']))
    n_events = si['n_bull'] + si['n_bear']
    
    best_dir_key = best['dir'].lower()
    con_5d = r.get(f'{best_dir_key}_5d_consistent', False)
    con_10d = r.get(f'{best_dir_key}_10d_consistent', False)
    
    genuine = abs(best['t']) > 1.96 and best['hit'] > 0.55 and best['n'] >= 10
    strong = (abs(best['t']) > 2.5 and best['hit'] > 0.65
              and best['n'] >= 15 and (con_5d or con_10d))
    marginal = abs(best['t']) > 1.5 and best['hit'] > 0.50 and best['n'] >= 5
    
    if strong:
        verdict = 'STRONG SIGNAL'
    elif genuine:
        verdict = 'GENUINE SIGNAL'
    elif marginal:
        verdict = 'MARGINAL — needs more data'
    else:
        verdict = 'NO SIGNAL'
    
    in_sample = ' (IN-SAMPLE)' if ccy == 'GBP' else ' (OUT-OF-SAMPLE)'
    tenors = f"{cfg['fast_tenor']}→{cfg['slow_tenor']}"
    print(f'\n  {r["pair"]} {tenors}{in_sample}')
    print(f'    Best: {best["dir"]} {best["h"]}: t={best["t"]:+.2f}, '
          f'hit={best["hit"]*100:.0f}%, '
          f'mean={best["bps"]:+.0f}bps, n={best["n"]}')
    print(f'    Events: {n_events} (bull={si["n_bull"]}, bear={si["n_bear"]})')
    print(f'    Sub-period: 5d={"YES" if con_5d else "NO"}, '
          f'10d={"YES" if con_10d else "NO"}')
    print(f'    Backtest: SR={bt["sharpe_net"]:.2f}, '
          f'Return={bt["total_return_net"]:.1f}%, '
          f'MaxDD={bt["max_drawdown"]:.1f}%')
    print(f'    >>> VERDICT: {verdict}')

# 2. The key finding
print('\n\n2. KEY FINDING: SIGNAL SPEED VARIES BY LIQUIDITY')
print('-'*70)
print('  The divergence signal generalises across major pairs,')
print('  but the optimal tenor pair differs by liquidity:')
print('    EUR (most liquid)  → 1W vs 1M  (info propagates fastest)')
print('    GBP (liquid)       → 1M vs 3M  (3-day lookback)')
print('    JPY (USD/xxx)      → 1M vs 3M  (7-day lookback, bear dir)')

# 3. Multiple testing
print('\n\n3. MULTIPLE TESTING')
print('-'*70)
n_tests = len(CURRENCIES) * 2 * 2
bonf = 0.05 / n_tests
n_surviving = sum(
    1 for ccy in CURRENCIES
    for d in ['bull_div', 'bear_div']
    for h in ['5d', '10d']
    if isinstance(results[ccy][d].get(h, {}).get('p_value', 1.0), float)
    and results[ccy][d].get(h, {}).get('p_value', 1.0) < bonf
)
print(f'  {n_tests} tests | Bonferroni threshold: p < {bonf:.4f}')
print(f'  Surviving: {n_surviving} / {n_tests}')
print(f'  NOTE: Per-pair tuning adds implicit degrees of freedom')
print(f'  beyond these {n_tests} tests. GBP is fully in-sample.')

# 4. Pooled analysis
print('\n\n4. POOLED CROSS-PAIR ANALYSIS')
print('-'*70)
for h in ['5d', '10d']:
    all_rets = []
    for ccy in CURRENCIES:
        r = results[ccy]
        ss = SPOT_SIGN[ccy]
        bull_rets = (data[ccy]['df']
                     .loc[r['bull_mask'], f'fwd_ret_{h}']
                     .dropna() * (+ss))
        bear_rets = (data[ccy]['df']
                     .loc[r['bear_mask'], f'fwd_ret_{h}']
                     .dropna() * (-ss))
        all_rets.extend([bull_rets, bear_rets])
    pooled = pd.concat(all_rets)
    if len(pooled) >= 3:
        t, p = sp_stats.ttest_1samp(pooled, 0)
        print(f'  {h}: n={len(pooled)}, '
              f'mean={pooled.mean()*10000:.1f}bps, '
              f'hit={((pooled>0).mean())*100:.0f}%, '
              f't={t:.2f}, p={p:.4f}')

# 5. Known risks
print('\n\n5. KNOWN RISKS')
print('-'*70)
print('  - GBP signal discovered and tuned in-sample')
print('  - EUR/JPY tenor pairs selected from a scan')
print('  - Small samples: n=23-35 events')
print('  - Look-ahead in quantile thresholds')
print('  - Regime dependence risk')

# 6. Position sizing
print('\n\n6. POSITION SIZING IMPLICATION')
print('-'*70)
for ccy in CURRENCIES:
    r = results[ccy]
    bt = bt_results[ccy]
    
    best_abs_t = max(
        abs(r['bull_div'].get(h, {}).get('t_stat', 0))
        for d in ['bull_div', 'bear_div']
        for h in ['5d', '10d']
    )
    
    if best_abs_t > 2.5 and bt['sharpe_net'] > 0.5:
        print(f'  {r["pair"]}: ALLOCATE')
    elif best_abs_t > 1.96 and bt['sharpe_net'] > 0:
        print(f'  {r["pair"]}: SMALL ALLOCATION')
    elif best_abs_t > 1.5:
        print(f'  {r["pair"]}: MONITOR ONLY')
    else:
        print(f'  {r["pair"]}: DO NOT ALLOCATE')

print('\n' + '='*80)